In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
from pathlib import Path
from pprint import pprint
import sys
from typing import Optional

if '..' not in sys.path: sys.path.append('..')

from datasets import load_dataset
from datasets.arrow_dataset import Dataset
import numpy as np
from matplotlib import pyplot as plt
from pydantic_yaml import parse_yaml_file_as
import torch
from torch import nn
import torch.nn.functional as F
from transformers import GPT2Tokenizer, AddedToken, PreTrainedTokenizer

from mllm.data.dsqrels import QrelsPlainBatch
from mllm.data.utils import load_qrels_datasets
from mllm.data.wiki.dswiki import WikiDsLoader
from mllm.exp.args import TOKENIZER_CFG_FNAME, RANKER_HG_MODEL_CFG_FNAME
from mllm.model.encdec_ranker_hg import RankerHg
from mllm.config.model import TokenizerCfg, RankerHgCfg
from mllm.tokenization.chunk_tokenizer import tokenizer_from_config, ChunkTokenizer
from s_03_06_train_ranker_hg_qrels import RankerCosEmbLoss



# RankerHg inference
## Config and paths

In [17]:
DATA_PATH = Path(os.path.expandvars('$HOME')) / 'data'
WIKI_DS_NAME = '20200501.en'
DS_MSMARCO_DIR_PATH = DATA_PATH / 'msmarco'
DS_FEVER_DIR_PATH = DATA_PATH / 'fever'

TRAIN_RANKER_HG_PATH = DATA_PATH / 'train_mllm_ranker_hg_qrels'
ranker_subdir = 'rankerhg-20250103_220003-msmarco-fever-inp128-pos_emb-lrs7x1-rdc_avg-step2-d512-h8-dbs_t-dmlp1024_512-tdec_t'
# ranker_subdir = 'rankerhg-20250104_093742-msmarco-fever-inp128-pos_emb-lrs7x1-rdc_avg-step2-d512-h8-dbs_t-dmlp512-tdec_t'

ranker_train_path = TRAIN_RANKER_HG_PATH / ranker_subdir
ranker_snapshot_fpath = ranker_train_path / 'best.pth'
ranker_model_cfg_fpath = ranker_train_path / RANKER_HG_MODEL_CFG_FNAME
ranker_tkz_cfg_fpath = ranker_train_path / TOKENIZER_CFG_FNAME

device_name = 'cpu'
# device_name = 'cuda'

device = torch.device(device_name)
print(device)

cpu


## Load models and datasets

In [18]:
model_cfg: RankerHgCfg = parse_yaml_file_as(RankerHgCfg, ranker_model_cfg_fpath)
pprint(model_cfg.dict())
inp_len = model_cfg.enc_pyr.inp_len

tkz_cfg: TokenizerCfg = parse_yaml_file_as(TokenizerCfg, ranker_tkz_cfg_fpath)
tkz = tokenizer_from_config(tkz_cfg)
ch_tkz = ChunkTokenizer(tkz_cfg.custom_tokens, tkz, n_emb_tokens=inp_len, fixed_size=True)
pad_tok = tkz_cfg.custom_tokens['pad'].ind

{'dec_rank': {'d_model': 512, 'mlp_sizes': [1024, 512], 'with_bias': True},
 'enc_pyr': {'d_inner': 2048,
             'd_k': 64,
             'd_model': 512,
             'd_v': 64,
             'dropout_rate': 0.0,
             'inp_len': 128,
             'n_heads': 8,
             'n_layers': 7,
             'n_similar_layers': 1,
             'pad_idx': 50267,
             'reduct_type': <HgReductType.Avg: 'avg'>,
             'step': 2,
             'vocab_encoder': {'d_model': 512,
                               'd_word_vec': 512,
                               'dropout_rate': 0.0,
                               'inp_len': 128,
                               'n_vocab': 50271,
                               'pad_idx': 50267,
                               'pos_enc_type': <PosEncType.Emb: 'emb'>}}}


In [19]:
chkpt = torch.load(ranker_snapshot_fpath, map_location=device)
model = RankerHg(model_cfg).to(device)
strict = True
# strict = False
model.load_state_dict(chkpt['model'], strict=strict)
model.eval()

RankerHg(
  (enc_pyr): EncoderPyramid(
    (vocab_encoder): VocabEncoder(
      (src_word_emb): Embedding(50271, 512, padding_idx=50267)
      (position_enc): Embedding(128, 512)
      (dropout): Dropout(p=0.0, inplace=False)
      (layer_norm): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
    )
    (enc_layers): ModuleList(
      (0-6): 7 x EncoderLayer(
        (slf_attn): MultiHeadAttention(
          (w_qs): Linear(in_features=512, out_features=512, bias=False)
          (w_ks): Linear(in_features=512, out_features=512, bias=False)
          (w_vs): Linear(in_features=512, out_features=512, bias=False)
          (fc): Linear(in_features=512, out_features=512, bias=False)
          (attention): ScaledDotProductAttention(
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (dropout): Dropout(p=0.0, inplace=False)
          (layer_norm): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
        )
        (pos_ffn): PositionwiseFeedForward(
         

In [7]:
ds_qrels = load_qrels_datasets([DS_MSMARCO_DIR_PATH, DS_FEVER_DIR_PATH], ch_tkz, inp_len, device)
dsqids = ds_qrels.df_qs['dsqid']

Join datasets:
   Msmarco. Queries: 372206. Docs: 3213835. QueryDocRels: 372206
   Fever. Queries: 123142. Docs: 5416568. QueryDocRels: 156101


## Inference on Qrels batch

In [8]:
loss_fn = RankerCosEmbLoss()

In [9]:
batch_size = 10
ib = 0
batch_off = ib * batch_size
dsqids_batch = dsqids.iloc[batch_off:batch_off + batch_size]

batch: QrelsPlainBatch = ds_qrels.get_batch_plain_qids(dsqids_batch)

Token indices sequence length is longer than the specified maximum sequence length for this model (13645 > 10000). Running this sequence through the model will result in indexing errors


In [10]:
batch.df_qs

,qid,query,dsid,dsqid
dsqid,,,,
0,1185869,)what was the immediate impact of the success ...,1,0
1,1185868,_________ justice is designed to repair the ha...,1,1
2,1183785,elegxo meaning,1,2
3,645590,what does physical medicine do,1,3
4,186154,feeding rice cereal how many times per day,1,4
5,457407,most dependable affordable cars,1,5
6,441383,lithophile definition,1,6
7,683408,what is a flail chest,1,7
8,484187,put yourself on child support in texas,1,8


In [11]:
batch.df_docs

,did,offset,dsid,dsdid,text,title
dsdid,,,,,,
2376038,59219,16949087845,1,2376038,The Manhattan Project Introduction Importance ...,Introduction
100673,59235,722556427,1,100673,"""From Wikipedia, the free encyclopedianavigati...",Restorative justice
1201976,576811,8572285503,1,1201976,"John, Sermons 2001The Ministry of the Holy Spi...",The Ministry of the Holy Spirit
1232741,576840,8792350091,1,1232741,"""Looking for a Physiatrist? Find a Physiatrist...",What Is a Physiatrist?
403227,114789,2878807930,1,403227,"""FEEDING GUIDELINES AGES 4 TO 6 MONTHSSTARTING...",FEEDING GUIDELINES AGES 4 TO 6 MONTHS
958172,389790,6827227963,1,958172,"""Surmount the snow with 7 of the best used all...",Surmount the snow with 7 of the best used all-...
2470237,576852,17619697389,1,2470237,"""From Wikipedia, the free encyclopedianavigati...",Goldschmidt classification
1380756,576861,9848986903,1,1380756,Flail chest describes a situation in which a p...,Flail Chest
277403,275258,1973432619,1,277403,Welcome!Notice ×THE TEXAS OAG CHILD SUPPORT WE...,Welcome!


In [12]:
batch.df_qrels

,qid,did,dsid,dsqid,dsdid
dsqid,,,,,
0,1185869,59219,1,0,2376038
1,1185868,59235,1,1,100673
2,1183785,576811,1,2,1201976
3,645590,576840,1,3,1232741
4,186154,114789,1,4,403227
5,457407,389790,1,5,958172
6,441383,576852,1,6,2470237
7,683408,576861,1,7,1380756
8,484187,275258,1,8,277403


In [13]:
for query in batch.df_qs['query']:
    print(query)

)what was the immediate impact of the success of the manhattan project?
_________ justice is designed to repair the harm to victim, the community and the offender caused by the offender criminal act. question 19 options:
elegxo meaning
what does physical medicine do
feeding rice cereal how many times per day
most dependable affordable cars
lithophile definition
what is a flail chest
put yourself on child support in texas
what happens in a wrist sprain


In [14]:
batch.qrels_masks

array([[ True, False, False, False, False, False, False, False, False,
        False],
       [False,  True, False, False, False, False, False, False, False,
        False],
       [False, False,  True, False, False, False, False, False, False,
        False],
       [False, False, False,  True, False, False, False, False, False,
        False],
       [False, False, False, False,  True, False, False, False, False,
        False],
       [False, False, False, False, False,  True, False, False, False,
        False],
       [False, False, False, False, False, False,  True, False, False,
        False],
       [False, False, False, False, False, False, False,  True, False,
        False],
       [False, False, False, False, False, False, False, False,  True,
        False],
       [False, False, False, False, False, False, False, False, False,
         True]])

In [20]:
qs_toks, qs_masks, docs_toks, docs_masks, qrels_masks = batch.gen_tensors()
out_rank = model(docs_toks, qs_toks)
print(out_rank)

tensor([[0.9564, 0.8179, 0.9734, 0.9407, 0.9418, 0.9212, 0.9519, 0.9418, 0.9417,
         0.9647],
        [0.9242, 0.8432, 0.9094, 0.9219, 0.8818, 0.9046, 0.9345, 0.8927, 0.8809,
         0.9240],
        [0.8340, 0.6952, 0.7766, 0.7528, 0.8509, 0.7512, 0.7575, 0.7925, 0.8440,
         0.8069],
        [0.8796, 0.8250, 0.9056, 0.9206, 0.8545, 0.8829, 0.9278, 0.8987, 0.8658,
         0.9262],
        [0.9678, 0.9236, 0.9180, 0.9376, 0.9645, 0.9520, 0.9175, 0.9504, 0.9791,
         0.9353],
        [0.9088, 0.7996, 0.8540, 0.8343, 0.9505, 0.8586, 0.8301, 0.8931, 0.9341,
         0.8806],
        [0.9036, 0.7787, 0.9050, 0.8864, 0.9044, 0.8689, 0.9012, 0.9026, 0.8933,
         0.9317],
        [0.8338, 0.9151, 0.8203, 0.8626, 0.8408, 0.8751, 0.8473, 0.8864, 0.8500,
         0.8599],
        [0.9481, 0.8370, 0.9335, 0.9104, 0.9467, 0.9174, 0.9052, 0.9183, 0.9542,
         0.9131],
        [0.9112, 0.9407, 0.8765, 0.9138, 0.9219, 0.9274, 0.8931, 0.9447, 0.9326,
         0.9203]], grad_fn=<

In [21]:
loss_fn(out_rank, qrels_masks)

(tensor(0.4962, grad_fn=<DivBackward0>),
 tensor(0.1018, grad_fn=<DivBackward0>),
 tensor(0.8906, grad_fn=<DivBackward0>))